In [ ]:
import time
from zumi.zumi import Zumi
from zumi.util.screen import Screen
from zumi.util.vision import Vision
from zumi.util.camera import Camera 
from zumi.personality import Personality

import moving_functions as mf
import vision_functioons as vf
import utils


zumi =    Zumi()
camera    = Camera()
screen    = Screen()
vision    = Vision()
personality = Personality(zumi, screen)

In [ ]:
zumi.mpu.calibrate_MPU()

In [ ]:
log = {}

In [ ]:
log = {}
zumi.reset_gyro()
desired_angle = zumi.read_z_angle() 
number_of_objects = 0

log = utils.log_event('start')
log = utils.log_event('end_line')

try:
    while True:
        # Set the threshold for the IR sensors and the speed
        threshold = 50 
        speed = 5

        if utils.object_detected():
            zumi.stop()
            zumi.brake_lights_on()
            time.sleep(2)
            zumi.brake_lights_off()
            log = utils.log_event('object_detected')
            number_of_objects += 1
            zumi.play_note(1, 500) # 1 is note type (1 - 60), 500 is duration in ms
            screen.draw_text_center("Objects: " + str(number_of_objects)) # Display object count on screen
            # Wait until the object is removed
            print("Waiting for object to be removed...")
            while utils.object_detected():
                zumi.stop()
                time.sleep(0.1)
            log = utils.log_event('object_removed')
            print("Object removed. Resuming movement.")
            log = utils.log_event('qr_code_read')
            message = vf.read_qr_code(speed, number_of_objects, threshold)
            log = utils.log_event('qr_code_command: ' + str(message) + " done")
            

        # Read all IR sensor values
        front_right, bottom_right, back_right, bottom_left, back_left, front_left = zumi.get_all_IR_data()

        # Correction to line if one sensor is on the line and the other is off
        desired_angle = mf.line_correction(bottom_left, bottom_right, desired_angle, threshold)

        # Move forward with the corrected heading
        if bottom_left > threshold or bottom_right > threshold:
            zumi.go_straight(speed, desired_angle)
        else:
            log = utils.log_event('end_line')
            
            if (log['end_line'][-1] - log['end_line'][-2]).total_seconds() > 3:
                go_left = True
            
                log = utils.log_event('check_left')
                # Turn to check if left is line
                turned_left_angle = mf.turn_to_check('left')

                # Calculate angle if turn was too much or not enough
                desired_angle = mf.turning_correction(turned_left_angle, 90)

                front_right, bottom_right, back_right, bottom_left, back_left, front_left = zumi.get_all_IR_data()
            else:
                go_left = False
            if (bottom_left > threshold or bottom_right > threshold) and go_left:
                log = utils.log_event('move_left')
                mf.move_after_turning(speed, desired_angle)
            else:
                
                log = utils.log_event('check_right')
                # Turn to check if right is line
                turned_right_angle = mf.turn_to_check('right') 
                
                # Calculate angle if turn was too much or not enough
                desired_angle = mf.turning_correction(turned_right_angle, 180)

                front_right, bottom_right, back_right, bottom_left, back_left, front_left = zumi.get_all_IR_data()

                if bottom_left > threshold or bottom_right > threshold:
                    log = utils.log_event('move_right')
                    mf.move_after_turning(speed, desired_angle)
                else:
                    zumi.stop()
                    log = utils.log_event('stop')
                    utils.finish_with_180_turn()
                    log = utils.log_event('finish_with_180_turn')
                    break
finally:
    zumi.stop()
    log = utils.log_event('stop')
    #save_dict_to_csv(log)